In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os


In [3]:
# Dataset paths
train_dir = r"D:\university\cse400(A,B,C)\Dataset\datasetM\train"
test_dir = r"D:\university\cse400(A,B,C)\Dataset\datasetM\test"

# Parameters
img_height = 224
img_width = 224
batch_size = 32
num_classes = len(os.listdir(train_dir))  # Assuming each subdirectory is a class


In [4]:
# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Rescaling for test data
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Loading data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 25255 images belonging to 16 classes.
Found 17547 images belonging to 16 classes.


In [5]:
# Load ResNet152 with pretrained weights from ImageNet
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the base model layers to prevent them from being updated during training
base_model.trainable = False


In [ ]:
# Add custom classification layers
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


: 

In [ ]:
# Train the model
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10,
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data)
)


Epoch 1/10
790/790 [==============================] - 452s 560ms/step - loss: 2.5829 - accuracy: 0.1715 - val_loss: 2.6707 - val_accuracy: 0.1165
Epoch 2/10
790/790 [==============================] - 311s 394ms/step - loss: 2.5142 - accuracy: 0.1861 - val_loss: 2.6842 - val_accuracy: 0.1165
Epoch 3/10
790/790 [==============================] - 307s 389ms/step - loss: 2.5060 - accuracy: 0.1890 - val_loss: 2.6786 - val_accuracy: 0.1148
Epoch 4/10
 39/790 [>.............................] - ETA: 3:26 - loss: 2.5265 - accuracy: 0.1779

In [ ]:
# Unfreeze specific layers
base_model.trainable = True
for layer in base_model.layers[:-50]:  # Freeze the first 50 layers
    layer.trainable = False

# Recompile the model
model.compile(optimizer=Adam(learning_rate=0.00001),  # Use a lower learning rate for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10,
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data)
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_resnet152_model.h5', save_best_only=True, monitor='val_accuracy')

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=20,
    steps_per_epoch=len(train_data),
    validation_steps=len(test_data),
    callbacks=[early_stop, model_checkpoint]
)
